# Scrappe Lychee

Website link : https://lycheethelabel.com/

Goal : get all the info product + each review.

+ Gets blogs information about Asian Streetwear's trend

## Import

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent

## Fonctions 

In [4]:
def initialize_driver():
    ua = UserAgent()
    user_agent = ua.random

    options = Options()
    options.add_argument('--headless')
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Firefox(options=options)
    return driver

In [5]:
def click_button(driver, xpath):
    try:
        button = driver.find_element(By.XPATH, xpath)
        button.click()
        return True
    except NoSuchElementException:
        print(f"Le bouton avec l'xpath '{xpath}' est introuvable.")
        return False

In [15]:
def scrape_content(driver, xpath, content_type='text', max_retries=3, timeout=8):
    for retry in range(max_retries):
        try:
            # Trouver tous les éléments correspondant à l'XPath fourni
            elements = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, xpath))
            )

           
            # Extraire le contenu spécifié pour chaque élément
            if content_type == 'text':
                result = [element.text for element in elements]
            elif content_type == 'href':
                result = [element.get_attribute('href') for element in elements]
            elif content_type == 'src':
                result = [element.get_attribute('src') for element in elements]
            else:
                raise ValueError("Type de contenu non pris en charge. Utilisez 'text', 'href', ou 'src'.")

            return result

        except TimeoutException:
            print(f"Timeout de {timeout} secondes atteint. Réessai {retry + 1}/{max_retries}.")

        except NoSuchElementException:
            print(f"Élément non trouvé. Vérifiez votre XPath. Réessai {retry + 1}/{max_retries}.")

        except StaleElementReferenceException:
            print(f"Stale Element Reference Exception. L'élément n'est plus attaché au DOM. Réessai {retry + 1}/{max_retries}.")

        except Exception as e:
            print(f"Une erreur s'est produite : {str(e)}")

    print(f"Impossible de trouver et extraire le contenu après {max_retries} tentatives.")
    return []

In [7]:
def action_aleatoire(driver):
    actions_possibles = ["scroll_full", "scroll_half", "move_to_element"]
    action_choisie = random.choice(actions_possibles)

    # Enregistrer la position actuelle de la page
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    if action_choisie == "scroll_full":
        # Action aléatoire : Faites défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    elif action_choisie == "scroll_half":
        # Action aléatoire : Faites défiler la moitié de la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
    elif action_choisie == "move_to_element":
        # Action aléatoire : Bougez la souris vers un élément aléatoire (par exemple, div de class PriceInformation_classifiedPrice__b-Jae)
        wait = WebDriverWait(driver, 10)
        element_to_move = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'center')))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_move)

    # Revenir à la position enregistrée
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

## Scrappe

In [ ]:
# Initialize the driver
driver = initialize_driver()

# Load the page
driver.get("https://lycheethelabel.com/")

In [ ]:
Plan D'action :

-Blog : cliquer sur div class header-blog := aller dans le a de la div ( le href)

sur chaque page recuperer tout les hrefs de blogs se trouvant dans : div class article__inner container  puis  h2  puis balise a  := recup des hrefs

sur chaque page de blog .   titre = by_css_selector('h1[data-mce-fragment]')
                            partie = by_css_selector('h2[data-mce-fragment]')
                            paragraphe = by_css_selector('p[data-mce-fragment]')
                            img = by_css_selector('div[data-mce-fragment]') recup limage dedans img
                            si presence ul =  by_css_selector('ul[data-mce-fragment]') 
                        sur 14 pages ( possibiltés de threadings)
                    

les urls des pages son : vetements = https://lycheethelabel.com/collections/+ particules de parties  := exemple https://lycheethelabel.com/collections/tops pour les tops
                         https://lycheethelabel.com/pages pour les autres particules =: exemple https://lycheethelabel.com/pages/about-us
je vais lister les différentes possibilités et scrapper dedans. 
je vais regarder chaque nombre de page par page pour savoir combien scrapper ( facilité du  multithread)

Sur page produits :   chaque balise a  class product-link ont un href := recuperer tout les hrefs et naviguer dedans

dans feuille produit :
    nom produit : h1 class title := recup text
    prix : span class current-price theme-money := recup text
    nombre reviews : span class jdgm-prev-badge__text := recup text
    size = recupe le ul de class clickyboxes options--size . parcourir chaque li et recup leur text
    wishlist := span de class swym-fave-count := recup text
    img produit :=   dans div class thumbnails desktop-only on recupe les img class rimage__image fade-in lazyautosizes lazyloaded recuperer le srcset via get attribute
    img_reseau = img class snpt__img  := recuperer src
    detail = div class content tout le texte
    recommendation = div class title = get text
    
    recommendations img = dans chaque div class block-inner-inner on recupe chaque  img class rimage__image fade-in lazyautosizes lazyloaded = img srcet
    reviews := a class jdgm-rev__prod-link  :=  recup href
    y a  acceder := 
    un review = div class product-reviews__review-card  on itere dans chaque pour obtenir
    
    note general = span class metrics-widget__rating := text
    recup:= div review__timestamp := get text
    nom profil = a class vertical-middle := text 
    date_post= div class review__timestamp
    lien_profil =  a class vertical-middle := get attribute href
    image = img class lazy-img review-images__picture-img := srcset 
    note := div class star-rating review__rating := get aria-label
    titre post = strong class review__title  := get text
    commentaire_post = div class review__body  ; on prend le p dedans := get text
    
    à la fin une fois page comme faites := click sur span de class pagination__page-number pagination__page-number--icon material-icons pour aller page comme suivante du meme produit
    
            

In [ ]:
# ebauche scrip pour carousell reseau
# Replace 'next_button_xpath' with the XPath of the button to move to the next image
next_button_xpath = "//button[@id='nextButton']"

# Replace 'carousel_xpath' with the XPath of the element containing the carousel
carousel_xpath = "//div[@class='your-carousel-class']"

# Function to extract image URLs from the current carousel state
def extract_image_urls():
    carousel_element = driver.find_element_by_xpath(carousel_xpath)
    carousel_html = carousel_element.get_attribute('innerHTML')
    return [url.split('"')[1] for url in carousel_html.split('srcset="')[1:]]

# Click the next button and extract image URLs multiple times
num_iterations = 5  # Adjust the number of iterations as needed

for _ in range(num_iterations):
    # Extract and print image URLs
    image_urls = extract_image_urls()
    for index, image_url in enumerate(image_urls):
        print(f"Image {index + 1}: {image_url}")

    # Click the next button to move to the next set of images
    next_button = driver.find_element_by_xpath(next_button_xpath)
    ActionChains(driver).move_to_element(next_button).click().perform()

In [ ]:
# Initialize the driver
driver = initialize_driver()

try:
    
    driver.get("https://lycheethelabel.com/")

    # acces blog
    blog_xpath = ".//div[@class='header-blog']/a"
    blog_href = scrape_content(driver, blog_xpath, content_type='href')
    print(blog_href)
    if blog_href:
        # Cliquer sur le premier lien
        driver.get(blog_href[0])
    else:
        print("Aucun lien trouvé.")

   
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ".//div[@class='article__inner container']"))
    )

   
    for page_number in range(1, 15):
        # Construire l'URL de la page suivante
        page_url = f"https://lycheethelabel.com/blogs/blog?page={page_number}"

      
        driver.get(page_url)

        # Récup s liens (href) des blogs
        blog_hrefs = scrape_content(driver, "//div[@class='article__inner container']//h2/a", content_type='href')
        print(blog_hrefs)

        # Parcourir chaque lien 
        for href in blog_hrefs:
            # Charger la page du blog
            driver.get(href)

            
            titre = scrape_content(driver, "//h1[@data-mce-fragment]", content_type='text')
            partie = scrape_content(driver, "//h2[@data-mce-fragment]", content_type='text')
            paragraphe = scrape_content(driver, "//p[@data-mce-fragment]", content_type='text')
            img = scrape_content(driver, "//div[@data-mce-fragment]//img", content_type='src')

            
            ul_content = scrape_content(driver, "//ul[@data-mce-fragment]", content_type='text')

            
            print(f"Titre: {titre}")
            print(f"Partie: {partie}")
            print(f"Paragraphe: {paragraphe}")
            print(f"Image: {img}")
            print(f"Contenu de la liste ul: {ul_content}")
            print("\n" + "="*50 + "\n")

finally:
    # Fermer le navigateur à la fin
    driver.quit()

['https://lycheethelabel.com/blogs/blog']
['https://lycheethelabel.com/blogs/blog/rave-culture-in-korea', 'https://lycheethelabel.com/blogs/blog/how-to-prep-for-a-rave', 'https://lycheethelabel.com/blogs/blog/the-trendiest-japanese-streetwear-fashion-of-the-moment', 'https://lycheethelabel.com/blogs/blog/top-10-unique-asian-streetwear-for-women-online-shopping-sites', 'https://lycheethelabel.com/blogs/blog/most-popular-kpop-fashion-trends', 'https://lycheethelabel.com/blogs/blog/the-top-5-platforms-to-stay-updated-on-the-latest-k-pop-news', 'https://lycheethelabel.com/blogs/blog/bts-unveiled-a-comprehensive-guide-to-the-phenomenal-k-pop-sensation', 'https://lycheethelabel.com/blogs/blog/top-5-kpop-trends', 'https://lycheethelabel.com/blogs/blog/secret-k-pop-spots-seoul-korea', 'https://lycheethelabel.com/blogs/blog/iconic-kpop-boy-groups']
Titre: ['Korean Raves and Everything You Need to Know About It!']
Partie: ['A Historical Glimpse of Korean Rave Scene', 'Influences on Korean Rave C